# Homework 5

You need to edit this `.ipynb` file by replacing "# Your code", "# Your answer", etc., click "Restart & Run All" in Jupyter Notebook to generate your results, and download it as an `.html` file. Please submit your `.ipynb` and `.html` files (instead of a `.zip` file) on Moodle. If you have questions regarding the homework, please email the TA Saumil Shah (sashah8@ncsu.edu), or attend our office hours.

### Problem 1

In the lecture, we have applied support vector regression to the load forecasting problem:

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn import svm
from sklearn.pipeline import make_pipeline
from sklearn.compose import TransformedTargetRegressor
def mape(a, f):
    return np.mean(np.absolute((a - f) / a))
df = pd.read_csv('/content/bse_clean.csv', parse_dates=['Date'])
df['Day'] = df['Date'].dt.dayofweek
train = ((df['Date'].dt.year >= 2004) & (df['Date'].dt.year <= 2006))
test = ((df['Date'].dt.year >= 2007) & (df['Date'].dt.year <= 2008))
X = np.concatenate((OneHotEncoder(sparse_output=False).fit_transform(df[['Hour', 'Month', 'Day']]), df[['T']]), axis=1)
y = df['Load'].to_numpy()
model = TransformedTargetRegressor(make_pipeline(StandardScaler(), svm.SVR()), transformer=StandardScaler())
model.fit(X[train], y[train])
mape(y[test], model.predict(X[test]))

0.030042531883163495

To achieve a better accuracy, you need to tune the parameters in `svm.SVR()`. Your objective is simply to beat the performance of the default setting. As long as the following MAPE is smaller than the above MAPE, you're done.

In [2]:
# Your code
# Change the following line

C = 10
model = TransformedTargetRegressor(
    regressor=make_pipeline(StandardScaler(), svm.SVR(C=C)),
    transformer=StandardScaler()
)
# Keep the following lines
model.fit(X[train], y[train])
mape(y[test], model.predict(X[test]))

0.02912719042594846

### Problem 2

In the lecture, we have adopted $k$-means clustering for daily load profiles:

In [3]:
from sklearn.cluster import KMeans
df = pd.read_csv('/content/bse_clean.csv', parse_dates=['Date'])
df = df[df['Month'] == 10].copy()
df_wide = df.pivot(index='Date', columns='Hour', values='Load').to_numpy()
kmeans = KMeans(2, random_state=0)
kmeans.fit(df_wide)
print(kmeans.labels_)

[0 0 0 1 1 0 0 0 0 0 1 1 1 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0
 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 1 0 0 0
 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0
 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0
 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 1 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0
 0]


In the above results, `0` means predicting weekday and `1` means predicting weekend. Use the calendar information to calculate the accuracy of the prediction:

In [4]:
# Your code
from sklearn.metrics import accuracy_score
actual_labels = df['Date'].dt.dayofweek.apply(lambda x: 1 if x >= 5 else 0)
daily_labels = actual_labels.groupby(df['Date'].dt.date).first().to_numpy()
accuracy = accuracy_score(daily_labels, kmeans.labels_)
print(accuracy)

0.9731182795698925
